In [1]:
Dataset = "FoCus"                      # "Blended Skill Talk", "IT-ConvAI2" , "FoCus" 

_COT = ""                                   #  "",    "-COT"

SCORING_METHOD = "train"                      # "train", "avg",  "length_prior", "benchmark"

LLM_name = f"Qwen2-7B-Instruct-{SCORING_METHOD}"

In [2]:
LLM_name

'Qwen2-7B-Instruct-train'

In [3]:
import pandas as pd

ds = pd.read_csv(f'Raw Responses/{Dataset}/{LLM_name}{_COT}.csv')

print("Shape:", ds.shape)

print("\nMissing Values:\n", ds.isnull().sum())

ds

Shape: (2000, 2)

Missing Values:
 gen_response     0
response_time    0
dtype: int64


,gen_response,response_time
0,"{\n ""response"": ""Descent of the Ganges is a r...",4.212004
1,"{\n ""response"": ""Little Rock Central High Sch...",4.141517
2,"{\n ""response"": ""The Manimahesh Lake is indee...",4.168975
3,"{\n ""response"": ""Yes, the Camp Randall Stadiu...",4.286672
4,"{\n ""response"": ""Urdaibai Bird Center in Spai...",4.294671
...,...,...
1995,"{\n ""response"": ""The Pelican Island National ...",3.926378
1996,"{\n ""response"": ""The neighborhood today is qu...",4.008988
1997,"{\n ""response"": ""Yes, it's quite accessible! ...",3.260124
1998,"{\n ""response"": ""The Alhambra Creek is a beau...",4.147906


In [4]:
print(ds.iloc[167]["gen_response"])

nan


In [6]:
import re
import json
import pandas as pd

# Define functions
def find_first_valid_json(text):

    if not isinstance(text, str):
        return None


    json_objects = re.findall(r'\{.*?\}', text, re.DOTALL)
    for obj in json_objects:
        try:
            json_obj = json.loads(obj)
            if "response" in json_obj:  # Only check for "response"
                return json_obj
        except json.JSONDecodeError:
            continue
    return None

def get_response(text):
    if text is not None:  # Check if text is not None
        try:
            return text['response']
        except (ValueError, SyntaxError, KeyError):
            return None
    return None



# Replace empty strings in 'gen_response' with None
ds.loc[ds['gen_response'] == '', 'gen_response'] = None

# Apply the find_first_valid_json function
ds['gen_response'] = ds['gen_response'].apply(lambda x: find_first_valid_json(x))

# Convert gen_response to None if it's not a valid string
ds['gen_response'] = ds['gen_response'].apply(lambda x: None if pd.isna(x) or x == 'nan' or isinstance(x, float) else x)

# Extract 'response' from the JSON objects
ds['gen_response'] = ds['gen_response'].apply(lambda x: get_response(x))

# Keep the 'response_time' column unchanged
ds['response_time'] = ds['response_time']

# Define the new column order
new_column_order = ['gen_response', 'response_time']

# Reorder the columns
ds = ds[new_column_order]

print("\nMissing Values:\n", ds.isnull().sum())
print(ds.shape)
ds



Missing Values:
 gen_response     4000
response_time       0
dtype: int64
(4000, 2)


,gen_response,response_time
0,None,0.791476
1,None,0.757963
2,None,0.893476
3,None,0.740705
4,None,0.894334
...,...,...
3995,None,0.876975
3996,None,0.667874
3997,None,0.820739
3998,None,0.335340


In [37]:
ds.to_csv(f'Responses/{Dataset}/{LLM_name}{_COT}.csv', index=False)